In [1]:
import SAGA
from easyquery import Query, QueryMaker
from SAGA.database import FitsTable
from SAGA import HostCuts as H
from astropy.table import join
from SAGA.utils import join_str_arr

In [2]:
saga = SAGA.QuickStart()
saga.set_default_base_version("paper2")

SAGA `shared_dir` set to /home/yymao/Dropbox/Academia/Collaborations/SAGA
SAGA `local_dir`  set to /home/yymao/Documents/Research/SAGA


In [3]:
hosts = saga.host_catalog.load(include_stats=True, query="paper2_complete")
print(len(hosts))

36


In [4]:
hosts['phot_coverage'] = join_str_arr(
    np.where(Query('COVERAGE_SDSS > 0.85').mask(hosts), 'S', '-'),
    np.where(Query('COVERAGE_DES_DR1 > 0.85').mask(hosts), 'D', '-'),
    np.where(Query('(COVERAGE_DECALS_DR6 > 0.95) | (COVERAGE_DECALS_DR7 > 0.95)').mask(hosts), 'L', '-'),
)
hosts['sats_below'] = np.where(hosts['sats_Mr_limit'] == hosts['sats_total'], '', np.char.mod('(%d)', hosts['sats_total']-hosts['sats_Mr_limit']))
hosts["spec_coverage"] = 100*(1-hosts["paper2_need_spec"] / hosts["paper2_total"])
hosts['internal_name'] = np.where(hosts["SAGA_NAME"] != "", hosts["SAGA_NAME"], hosts["HOSTID"])

In [5]:
t = hosts['COMMON_NAME', 'internal_name', 'RA', "DEC", 'V_HELIO', 'DIST', 'K_ABS', 'phot_coverage', 'spec_coverage', 'sats_Mr_limit', 'sats_below', 'sats_missed_corrected']

t.sort('RA')

t['RA'].format = '%.3f'
t['DEC'].format = '%.3f'
t['V_HELIO'].format = '%d'
t['DIST'].format = '%.1f'
t['K_ABS'].format = '%.2f'
t['sats_missed_corrected'].format = '%.2f'
t['spec_coverage'].format = '%.1f'

In [6]:
colheads = [
    "Common\nName",
    "SAGA\nName",
    "R.A.\n[deg]",
    "Dec.\n[deg]",
    "HRV\n[\\kms]",
    "$D$\n[Mpc]",
    "$M_{K,o}$\n[mag]",
    "Phot.\nCoverage",
    "Spec.\nCoverage (\%)",
    "Confirmed\n$N_\\text{sat}$",
    "Miss.\n$N_\\text{sat}$"
]

In [7]:
colheads_formatted = [[], [], []]
for i, colhead in enumerate(colheads):
    colhead = colhead.splitlines()
    while len(colhead) < 2:
        colhead.insert(0, "~")
    colhead.append(f"({i+1})")
    for new, l in zip(colheads_formatted, colhead):
        new.append(l)

In [8]:
print("\\\\[-6pt]\n".join((" & ".join((f"\colhead{{{i}}}" for i in row)) for row in colheads_formatted)))

\colhead{Common} & \colhead{SAGA} & \colhead{R.A.} & \colhead{Dec.} & \colhead{HRV} & \colhead{$D$} & \colhead{$M_{K,o}$} & \colhead{Phot.} & \colhead{Spec.} & \colhead{Confirmed} & \colhead{Miss.}\\[-6pt]
\colhead{Name} & \colhead{Name} & \colhead{[deg]} & \colhead{[deg]} & \colhead{[\kms]} & \colhead{[Mpc]} & \colhead{[mag]} & \colhead{Coverage} & \colhead{Coverage (\%)} & \colhead{$N_\text{sat}$} & \colhead{$N_\text{sat}$}\\[-6pt]
\colhead{(1)} & \colhead{(2)} & \colhead{(3)} & \colhead{(4)} & \colhead{(5)} & \colhead{(6)} & \colhead{(7)} & \colhead{(8)} & \colhead{(9)} & \colhead{(10)} & \colhead{(11)}


In [9]:
from io import StringIO
o = StringIO()
t.write(o, format="ascii.aastex")
o.seek(0)

for line in o:
    line = line.strip()
    if line[0] != "\\":
        print(line)

UGC00903 & HarryPotter & 20.449 & 17.592 & 2516 & 38.4 & -23.54 & S-L & 92.4 & 4 &  & 0.46 \\
NGC1015 & Narnia & 39.548 & -1.319 & 2625 & 37.0 & -23.50 & SDL & 95.9 & 2 &  & 0.60 \\
NGC1309 & nsa133355 & 50.527 & -15.400 & 2136 & 34.3 & -23.57 & -DL & 89.7 & 3 &  & 0.97 \\
PGC013646 & nsa133549 & 55.734 & -12.916 & 2163 & 31.9 & -23.14 & -DL & 82.9 & 3 &  & 1.15 \\
ESO121-026 & pgc18880 & 95.412 & -59.740 & 2266 & 35.0 & -23.71 & -D- & 85.6 & 2 &  & 0.12 \\
NGC2543 & StarTrek & 123.241 & 36.255 & 2470 & 37.6 & -23.46 & S-L & 91.7 & 2 &  & 1.00 \\
UGC04906 & nsa16235 & 139.416 & 52.993 & 2273 & 36.1 & -23.36 & S-L & 81.1 & 6 &  & 2.54 \\
NGC2962 & nsa157374 & 145.225 & 5.166 & 1958 & 34.8 & -24.05 & S-L & 91.9 & 4 &  & 1.33 \\
NGC2967 & nsa32 & 145.514 & 0.336 & 1892 & 29.9 & -23.51 & S-L & 88.8 & 1 & (1) & 0.69 \\
NGC3689 & nsa139467 & 172.046 & 25.661 & 2737 & 39.8 & -23.80 & S-L & 93.8 & 2 &  & 0.98 \\
NGC3976 & nsa140458 & 178.989 & 6.750 & 2496 & 35.9 & -24.00 & S-L & 82.2 & 2 &  &